## DeChallenge

Autor: Andres Angarita
Última actualización: Abril 2023

En este notebook se presenta un análisis exploratorio de datos sobre las monedas del mercado utilizando la api de coingecko.

El objetivo de este análisis es identificar ciertos campos del top 120 de las monedas del mercado.

Se utilizó el conjunto de datos markets que se puede obtener con que querystring de la primera seccion.

El análisis se divide en las siguientes secciones:
- Sección 1: Obtencion de datos
- Sección 2: Tratamiento de datos
- Sección 3: Exporte de datos

Nota: Este notebook fue creado utilizando jupyter 1.0.0.


### Seccion 1. Obtencion de datos

In [46]:
import requests
import traceback
import sys
import pandas as pd

try:
    markets = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=120&page=1&sparkline=false&locale=en')
    cols=['Id','Name','Current_price','Ath','Ath_date','Market_cap','Market_cap_rank','Circulating_supply','Max_supply','Last_updated','Description_en','Image_large','Links']
    filtered_markets = pd.DataFrame(columns=cols)
    
    if markets.status_code == 200:
        data = markets.json()
        data_dict = {}
        
        for coin in data:
            data_dict['Id'] = coin['id']
            data_dict['Name'] = coin['name']
            data_dict['Current_price'] = coin['current_price']
            data_dict['Ath'] = coin['ath']
            data_dict['Ath_date'] = coin['ath_date']
            data_dict['Market_cap'] = coin['market_cap']
            data_dict['Market_cap_rank'] = coin['market_cap_rank']
            data_dict['Circulating_supply'] = coin['circulating_supply']
            data_dict['Max_supply'] = coin['max_supply']
            data_dict['Last_updated'] = coin['last_updated']
            
            additional_info = requests.get('https://api.coingecko.com/api/v3/coins/'+coin['id'])
            if additional_info.status_code == 200:
                more_info = additional_info.json()
                description_en = more_info["description"]["en"]
                links = more_info["links"]["homepage"]
            
            data_dict['Description_en'] = description_en
            
            img = requests.get(coin['image'], stream=True)
            data_dict['Image_large'] =sys.getsizeof(img.content)
            
            data_dict['Links'] = links[0]
    
            tmpDf = pd.DataFrame.from_dict(data_dict, orient='index').T
            filtered_markets = pd.concat([filtered_markets,tmpDf],ignore_index=True)
    else:
        print('No se pudo obtener los datos de CoinGecko')
except Exception as e:
    print('Error')
    print("Ocurrió un error:", e)
    traceback.print_exc()


### Seccion 2. Tratamiento de datos

In [34]:
from bs4 import BeautifulSoup as bs
from dateutil.parser import parse

def get_exchange(currency="USD", amount=1):
    '''funcion que obtiene el valor del dolar en pesos colombianos 
    Parameters
    ----------
    currency : string
        descripcion de la moneda a convertir, teniendo en cuenta la nomenclatura de la pagina x-rates.com 
    amount : float
        cantidad de moneda a convertir a pesos colombianos
    Returns
    -------
    float 
        valor de la moneda en pesos colombianos
    '''
    
    content = requests.get(f"https://www.x-rates.com/table/?from={currency}&amount={amount}").content
    soup = bs(content, "html.parser")
    # Obtener fecha
    #price_datetime = parse(soup.find_all("span", attrs={"class": "ratesTimestamp"})[1].text)
    exchange_tables = soup.find_all("table")
    exchange_rates = {}
    
    for exchange_table in exchange_tables:
        for tr in exchange_table.find_all("tr"):
            #filas de la tabla
            tds = tr.find_all("td")
            if tds:
                currency = tds[0].text
                exchange_rate = float(tds[1].text)
                exchange_rates[currency] = exchange_rate  
    return exchange_rates["Colombian Peso"]


In [47]:
filtered_markets.head(20)

,Id,Name,Current_price,Ath,Ath_date,Market_cap,Market_cap_rank,Circulating_supply,Max_supply,Last_updated,Description_en,Image_large,Links
0,bitcoin,Bitcoin,27512,69045,2021-11-10T14:24:11.849Z,532480341442,1,19355550.0,21000000.0,2023-04-25T00:06:32.137Z,Bitcoin is the first successful internet money...,9441,http://www.bitcoin.org
1,ethereum,Ethereum,1842.12,4878.26,2021-11-10T14:24:19.604Z,221803547014,2,120409031.538542,None,2023-04-25T00:06:27.802Z,"Ethereum is a global, open-source platform for...",10010,https://www.ethereum.org/
2,tether,Tether,1.0,1.32,2018-07-24T00:00:00.000Z,81577575491,3,81538750252.410507,None,2023-04-25T00:05:00.332Z,Tether (USDT) is a cryptocurrency with a value...,10004,https://tether.to/
3,binancecoin,BNB,331.09,686.31,2021-05-10T07:24:17.097Z,52278083304,4,157886280.0,200000000.0,2023-04-25T00:06:25.572Z,Binance Coin is the cryptocurrency of the <a h...,17366,https://www.binance.com/
4,usd-coin,USD Coin,0.999462,1.17,2019-05-08T00:40:28.300Z,30728183094,5,30748436106.8381,None,2023-04-25T00:06:25.258Z,USDC is a fully collateralized US dollar stabl...,13168,https://www.circle.com/en/usdc
5,ripple,XRP,0.460496,3.4,2018-01-07T00:00:00.000Z,23878162114,6,51750810378.0,100000000000.0,2023-04-25T00:06:32.334Z,Ripple is the catchall name for the cryptocurr...,9610,https://ripple.com/currency/
6,cardano,Cardano,0.383643,3.09,2021-09-02T06:00:10.474Z,13453311209,7,35045020830.323402,45000000000.0,2023-04-25T00:06:25.060Z,,39337,https://www.cardano.org/en/home/
7,staked-ether,Lido Staked Ether,1839.77,4829.57,2021-11-10T14:40:47.256Z,11294852161,8,6140200.895841,6140200.895841,2023-04-25T00:06:30.586Z,Lido Staked Ether (stETH) is a token that repr...,14108,https://www.lido.fi
8,dogecoin,Dogecoin,0.078895,0.731578,2021-05-08T05:08:23.458Z,10963806341,9,139076666383.704987,None,2023-04-25T00:06:25.785Z,Dogecoin is a cryptocurrency based on the popu...,48733,http://dogecoin.com/
9,matic-network,Polygon,0.989312,2.92,2021-12-27T02:08:34.307Z,9156699473,10,9249469069.284929,10000000000.0,2023-04-25T00:06:27.964Z,Polygon (Previously Matic Network) is the firs...,8994,https://polygon.technology/


from bs4 import BeautifulSoup as bs
from dateutil.parser import parse
from pprint import pprint

